## Deskriptive Betrachtung der Quelle
Dependenzen und Daten laden:

In [1]:
from pathlib import Path
from pandas import DataFrame
import pandas as pd

In [2]:
def read_csv(path: Path) -> DataFrame:
    df = pd.read_csv(path)
    return df

In [3]:
df = read_csv("../data/all.csv")


Ermitteln wie Oft die Orte als Absende Orte vorliegen

In [4]:
from aggregate.letter_map import letter_map, places_set

# alle einzelnen Absendeorte initialisieren
absender_set = set()
for item, val in letter_map.items():
    absender_set.add(val[1])
# Zählen der Absendeorte intitialisieren
absender_count = {}
for i in absender_set:
    absender_count[i] = 0
    
# Orte zählen
for item, val in letter_map.items():
    absender_count[val[1]] += 1
    
absender_count

{'Clausthal': 1,
 'Ferrara': 1,
 'Wien': 27,
 'Göhrde': 3,
 'Braunschweig': 8,
 'Leipzig': 1,
 'Oranienburg': 1,
 'Berlin': 62,
 'Helmstedt': 1,
 'Bruchhausen': 1,
 'Hannover': 173,
 'Zellerfeld': 1,
 'Rehburg': 1,
 'Wolfenbüttel': 14,
 'Karlsbad': 1,
 'Modena': 1,
 'Regensburg': 1,
 'Wolfenbüttel oder Helmstedt': 1,
 'Celle': 3,
 'Linsburg': 11,
 'Ebstorf': 6,
 'Torgau': 1,
 'München': 1,
 'Herrenhausen': 53,
 'Lietzenburg': 8}

testen, ob die Anzahl der Absendeorte mit der Anzahl der Briefe übereinstimmt:

In [5]:

absolute_count = 0
for item, val in absender_count.items():
    absolute_count += val
print(absolute_count, len(letter_map))

382 382


### 1. Zum Address-Verhalten
Leibnitz neigt dazu den Absendeort, wenn überhaupt, unten im Brief einzufügen. 
Sophie von Hannover ist deutlich sorgfältiger und setzt oben meist den Absende Ort ein und schließt mit "An Herrn Leibnitz in {Leibnitzs Aufenthaltsort}"

Auslesen, wie hoch der Impact von Absendeort auf Ortsnennung ist:

In [6]:
# Dataframe mit Absendeorten:
absendeorte = df[df['Place'].isin(absender_set)]
ab_all = set(absendeorte['Place'])

missing_places = absender_set - ab_all
print( missing_places)

{'Lietzenburg', 'Rehburg', 'Wolfenbüttel oder Helmstedt'}


## 2. Reise Verhalten

TODO: Karte machen?

In [7]:
for v in missing_places:
    print(v, v in places_set)

Lietzenburg True
Rehburg True
Wolfenbüttel oder Helmstedt False


## 3. Generelles Briefverhalten
Buchstaben gesamt:
* Sophie: 315884
* Leibnitz: 908847

Wörter gesamt: 
* Sophie: 45842
* Leibnitz: 125487

In [1]:
from aggregate.letter_map import letter_map
#read in data
import pandas as pd
places_sophie = pd.read_csv("../data/per person/Sophie.csv")
places_leibnitz = pd.read_csv("../data/per person/Leibnitz.csv")

In [19]:
# descriptive analysis about letters
from pathlib import Path

total_characters_sophie = 0
total_characters_leibnitz = 0
total_words_sophie = 0
total_words_leibnitz = 0

paths = [path for path in Path("../letters/cleaned").iterdir() if path.is_file()]

for path in paths:
    letter_num = int(path.stem.split("_")[-1])
    active_person = letter_map[letter_num][0]
    with open(path, "r") as f:
        body = f.read()
        num_chars = len(body)
        num_words = len(body.split(" "))
        if active_person == "Sophie":
            total_characters_sophie += num_chars
            total_words_sophie += num_words
        elif active_person == "Leibnitz":
            total_characters_leibnitz += num_chars
            total_words_leibnitz += num_words
        else:
            print(f"found rouge letter: {letter_num}")
        

In [20]:
print(total_characters_sophie, total_words_sophie)
print(total_characters_leibnitz, total_words_leibnitz)

315884 45842
908847 125487


### 3.1 Anzahl Briefe
* Sophie: 171
* Leibnitz: 211

In [2]:
# amount of letters per person
## Sophie
num_letters_sophie = 0
## Leibtnitz
num_letters_leibnitz = 0

for item, val in letter_map.items():
    if val[0] == "Sophie":
        num_letters_sophie += 1
    elif val[0] == "Leibnitz":
        num_letters_leibnitz +=1
    else:
        print(f"rouge letter found: {item}")

print(num_letters_sophie, num_letters_leibnitz)        

171 211


In [21]:
# characters/ words per letter:
characters_per_letter_sophie = total_characters_sophie/num_letters_sophie
words_per_letter_sophie = total_words_sophie / num_letters_sophie

characters_per_letter_leibnitz = total_characters_leibnitz/num_letters_leibnitz
words_per_letter_leibnitz = total_words_leibnitz / num_letters_leibnitz

print(f"sophie: characters per letter: {characters_per_letter_sophie} | words per letter: {words_per_letter_sophie}")
print(f"leibnitz: characters per letter: {characters_per_letter_leibnitz} | words per letter: {words_per_letter_leibnitz}")



sophie: characters per letter: 1847.2748538011697 | words per letter: 268.08187134502924
leibnitz: characters per letter: 4307.331753554502 | words per letter: 594.7251184834123


sophie: characters per letter: 1847.2748538011697 | words per letter: 268.08187134502924

leibnitz: characters per letter: 4307.331753554502 | words per letter: 594.7251184834123

### 3.2 Orts Dichte in Briefen

insgesamt:
* Sophie: 1178
* Leibnitz: 2231

pro brief:
* Sophie: 6.888888888888889
* Leibnitz: 10.57345971563981

Verhältnis von Ort pro Wort in Brief:
* Sophie: 38.9151103565365
* Leibnitz: 56.24697445091887
 

In [6]:
# total number of letters per person
total_letters_sophie = sum(places_sophie['Count'])
total_letters_leibnitz = sum(places_leibnitz['Count'])
print(total_letters_sophie, total_letters_leibnitz)

In [9]:
# place density per letter per person
density_sophie = total_letters_sophie / num_letters_sophie
density_leibnitz = total_letters_leibnitz / num_letters_leibnitz
print(density_sophie, density_leibnitz)

6.888888888888889 10.57345971563981


In [22]:
# place per word per letter 
place_per_word_sophie =  words_per_letter_sophie / density_sophie
place_per_word_leibnitz = words_per_letter_leibnitz / density_leibnitz

print(place_per_word_sophie, place_per_word_leibnitz)

38.9151103565365 56.24697445091887


### 3.3 Vergleich ob person Orte öfter nennt (normalisiert/nicht normalisiert)